### Include Header Files

In [28]:
import os
import subprocess
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import re 
import random

# Keras 2.2.4/tensorflow 1.14.0 -> Version matching for local machine

from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras import initializers



### Read the csv files of Passes

In [29]:
txtfile = "passes_name_8.csv"
import csv
with open(txtfile,'rt')as f:
  data = csv.reader(f)
  for row in data:
        print("row")
passes=row
# print(len(passes))
# print(passes)
cplsryPass=["mem2reg","loop-simplify","loop-rotate","indvars"]

row


### This function generates set of passes for KTransFormations

In [30]:
def genPasses(KTfms,maxPasses):
    passList = []
    # Number of Transformation Per Program
    for i in range(KTfms):
        passCount = random.randint(1,maxPasses)
        passString = ""
        uniqueCheck=[]
        # Number of Passes in each Transformation
        for j in range(passCount):
            passNo = random.randint(0,len(passes)-1)
            
            # Not compulsary pass.
            if passes[passNo] not in cplsryPass:
                if passes[passNo] not in uniqueCheck:
                    uniqueCheck.append(passes[passNo])
                    
                    # Add all compulsary passes.
                    for cpPass in cplsryPass:
                        if cpPass not in uniqueCheck:
                            # If New add in string and uniqueCheckList.
                            uniqueCheck.append(cpPass)
                            
            # 4 Compulsary pass ["mem2reg","loop-simplify","loop-rotate","indvars"].
            else:
                if passes[passNo] == "mem2reg" and passes[passNo] not in uniqueCheck:
                    uniqueCheck.append(passes[passNo])
                    
                if passes[passNo] == "loop-simplify" and passes[passNo] not in uniqueCheck:
                    uniqueCheck.append(passes[passNo])
                    
                if passes[passNo] == "loop-rotate" and passes[passNo] not in uniqueCheck:
                    uniqueCheck.append(passes[passNo])
                    
                    # Adding passes required for Loop rotate pass.
                    if "loop-simplify" not in uniqueCheck:
                        uniqueCheck.append("loop-simplify")
                        
                if passes[passNo] == "indvars" and  passes[passNo] not in uniqueCheck:
                    uniqueCheck.append(passes[passNo])
                    
                    # Adding passes required for indvars pass.
                    if "mem2reg" not in uniqueCheck:
                        uniqueCheck.append("mem2reg")
                    if "loop-simplify" not in uniqueCheck:
                        uniqueCheck.append("loop-simplify")
                    if "loop-rotate" not in uniqueCheck:
                        uniqueCheck.append("loop-rotate")
#         print(uniqueCheck)
        passList.append(uniqueCheck)
    return passList

### Function to generate opt command and run on passes

In [31]:
def genVector(LL_FILE_ip,KTfms,maxPasses):
    PATH = "/home/gagandeep/Desktop/work/polybench/Utilities_IR2Vec/Utilities/"
    PATH_OPT = PATH + "opt"
    PATH_SO_DIR = PATH + "IR2Vec-RD.so"
    PATH_EMBD_DIR = PATH + "embedding_300D_TransE_1500ep_Quant.txt"
#     LL_FILE_PATH = "/home/gagandeep/Desktop/work/polybench/pb_ll/2mm/O0/"
#     LL_FILE_NAME = "O0_2mm.ll"
#     LL_FILE = LL_FILE_PATH + LL_FILE_NAME
    # LL_FILE_ip is a pair with location and filename of LL-file.
    LL_FILE = os.path.join(LL_FILE_ip[0], LL_FILE_ip[1])
    OP_FILE_NAME_txt = "vectors/" + LL_FILE_ip[1]+".txt"
    cmd=[PATH_OPT,"-load",PATH_SO_DIR,"-IR2Vec_RD","-file",PATH_EMBD_DIR,
               "-bpi","1","-class","-1","-of",OP_FILE_NAME_txt,"-level","f",
               "-wo","1","-wt","0.5","-wa","0.2",LL_FILE]

    transCmd = []
    for passSet in genPasses(KTfms,maxPasses):
        transCmd = list(cmd)        
        for eachPass in passSet:
            transCmd.append("-"+eachPass)
    #     print(transCmd)
        temp = subprocess.Popen(transCmd, stdout = subprocess.PIPE)

    # output = str(temp.communicate()) 

    # output = output.split("\n") 
    # output = output[0].split('\\')
    # # for val in output:
    # #     print(val)

    # res = [] 

    #     # iterate through the output 
    #     # line by line 
    # for line in output: 
    #     res.append(line) 

    # # print the output 
    # for i in range(1, len(res) - 1): 
    #     print(res[i]) 

### Generate vectors for all LL-files

In [32]:
def call2genVec():
    llFiles = []
    for dirname, _, filenames in os.walk('./ll_files'):
        for filename in filenames:
            llFiles.append([dirname,filename])

    for eachFile in llFiles:
        genVector(eachFile,10,15)
        print(os.path.join(val[0], val[1]))

./ll_files/adc.ll
./ll_files/adc.ll
./ll_files/adc.ll
./ll_files/adc.ll


### Preprocess the vectors to create dataset

In [97]:
IRVec = []
vecFiles = []
class_label=0
for dirname, _, filenames in os.walk('./vectors'):
    for filename in filenames:
        vecFiles.append([dirname,filename])
for eachFile in vecFiles:
    txtfile = os.path.join(eachFile[0], eachFile[1])
    traindf = pd.read_csv(txtfile,delimiter="\t",header=None)
    traindf = traindf.dropna(axis=1)
#     traindf = traindf.drop(traindf.columns[:2],axis=1)
#     print(traindf.shape[0])
#     print(traindf.iloc[199,:])
    # Count of Different Functions
    functCount = -1
    for index,Count in enumerate(traindf.iloc[:,0]):
        if Count == traindf.iloc[0,0]:
            if functCount == -1:
                functCount = index
            else:
                functCount = index
                break
#     print(functCount)
    traindf = traindf.drop(traindf.columns[:2],axis=1)
#     print(traindf.iloc[199,:])
#     print(type(traindf.iloc[]))
    for index,dataPoint in enumerate(traindf.iloc[:]):
#         new_data_point = dataPoint.values.reshape(1,dataPoint.size)
#         new_data_point=pd.DataFrame(dataPoint)
#         print(traindf.iloc[dataPoint,:])
        new_data_point = traindf.iloc[dataPoint,:]
        new_data_point = new_data_point.values.reshape(1,new_data_point.size)
        new_data_point= pd.DataFrame(new_data_point)
        
        if index == 0:
            temp = new_data_point
#                 labels.append(class_label)

        else :
            temp = pd.concat([temp, new_data_point])
#                 labels.append(class_label)
    
        print(temp)
        break
        
#         print(new_data_point)
#         print(index)

        0         1         2         3         4         5         6    \
0 -1.822447 -1.919357  0.901833 -4.671315 -0.961689  1.243988  0.938743   

        7         8         9    ...       290       291       292       293  \
0 -0.440845  0.987527 -2.442327  ...  1.145447  2.829992 -0.443599 -0.744863   

        294       295      296       297       298       299  
0 -0.669255  0.450204  1.32371  1.607742 -0.243377  0.281935  

[1 rows x 300 columns]


In [30]:
a = ""
tra = ['/home/gagandeep/Desktop/work/polybench/Utilities_IR2Vec/Utilities/opt', '-load', '/home/gagandeep/Desktop/work/polybench/Utilities_IR2Vec/Utilities/IR2Vec-RD.so', '-IR2Vec_RD', '-file', '/home/gagandeep/Desktop/work/polybench/Utilities_IR2Vec/Utilities/embedding_300D_TransE_1500ep_Quant.txt', '-bpi', '1', '-class', '-1', '-of', 'O0_2mm.ll.txt', '-level', 'f', '-wo', '1', '-wt', '0.5', '-wa', '0.2', '/home/gagandeep/Desktop/work/polybench/pb_ll/2mm/O0/O0_2mm.ll', '-gvn', '-mem2reg', '-loop-simplify', '-loop-rotate', '-indvars', '-bb-vectorize', '-loop-extract-single', '-aggressive-instcombin', '-loop-deletion', '-ipsccp', '-inline']
for val in tra:
    a = a + val +" "
print(a)

/home/gagandeep/Desktop/work/polybench/Utilities_IR2Vec/Utilities/opt -load /home/gagandeep/Desktop/work/polybench/Utilities_IR2Vec/Utilities/IR2Vec-RD.so -IR2Vec_RD -file /home/gagandeep/Desktop/work/polybench/Utilities_IR2Vec/Utilities/embedding_300D_TransE_1500ep_Quant.txt -bpi 1 -class -1 -of O0_2mm.ll.txt -level f -wo 1 -wt 0.5 -wa 0.2 /home/gagandeep/Desktop/work/polybench/pb_ll/2mm/O0/O0_2mm.ll -gvn -mem2reg -loop-simplify -loop-rotate -indvars -bb-vectorize -loop-extract-single -aggressive-instcombin -loop-deletion -ipsccp -inline 
